<a href="https://colab.research.google.com/github/huune2k/BTVN_AI_8-8-2022/blob/AI_8-5-2022/Huu's_face_train_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from tensorflow.keras.optimizers import RMSprop
from keras.backend import categorical_crossentropy

from keras.utils import np_utils
import cv2 as cv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/Huu_FaceData

test  train


In [ ]:
import tensorflow as tf
from pathlib import Path
train_root = Path('/content/drive/MyDrive/Huu_FaceData/train')
Huu_dir = train_root/'HuuFace'
NoneHuu_dir = train_root/'NoneHuu'

tf.io.gfile.mkdir(Huu_dir)
tf.io.gfile.mkdir(NoneHuu_dir)

for item in train_root.glob("*.jpg"):
  cur_dir = train_root / item.name
  if item.name[1] == 'H':
    des_dir = Huu_dir / item.name
  else:
    des_dir = NoneHuu_dir / item.name
  tf.io.gfile.copy(cur_dir, des_dir, overwrite=True)
  tf.io.gfile.remove(cur_dir)

In [ ]:
!ls /content/drive/MyDrive/Huu_FaceData/train/HuuFace| wc -l
!ls /content/drive/MyDrive/Huu_FaceData/train/NoneHuu| wc -l

60
60


In [ ]:
#build input pipeline with keras
batch_size = 2
img_height = 150
img_width = 150

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_root,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size = batch_size    
)

Found 120 files belonging to 2 classes.
Using 96 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    train_root,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size = batch_size    
)

Found 120 files belonging to 2 classes.
Using 24 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['HuuFace', 'NoneHuu']


In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10,10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax= plt.subplot(3,3,i+1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
# plt.axis("off")

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(150,150,3)))
model.add(Conv2D(32,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128,(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(32,activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1,activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 150, 150, 32)      896       
                                                                 
 conv2d_25 (Conv2D)          (None, 150, 150, 32)      9248      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 75, 75, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 75, 75, 64)        18496     
                                                                 
 conv2d_27 (Conv2D)          (None, 75, 75, 64)        36928     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 37, 37, 64)       0         
 g2D)                                                 

In [ ]:
# compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# train model
model.fit(train_ds, validation_data=val_ds, epochs=5)


Epoch 1/5
48/48 [==============================] - 2s 12ms/step - loss: nan - accuracy: 0.5312 - val_loss: nan - val_accuracy: 0.2917
Epoch 2/5
48/48 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.5521 - val_loss: nan - val_accuracy: 0.2917
Epoch 3/5
48/48 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.5521 - val_loss: nan - val_accuracy: 0.2917
Epoch 4/5
48/48 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.5521 - val_loss: nan - val_accuracy: 0.2917
Epoch 5/5
48/48 [==============================] - 1s 10ms/step - loss: nan - accuracy: 0.5521 - val_loss: nan - val_accuracy: 0.2917


In [ ]:
# import pickle
# file_name = "HuuFace.sav"
# pickle.dump(model, open(file_name, 'wb'))
model.save('Huuface1.h5')